In [ ]:
import pandas as pd
import numpy as np
import os
from openpyxl import Workbook
import datetime

In [ ]:
# Files Directions, Rutes and file names
filepathCostos = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\01. Costos'
filepathOtAv = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\04. OT y Av'
filepathReporte = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\08. Reportes\Reporte de Costes'
filepathMateriales = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\03. Materiales'


#File Names
CeCo = r'CeCo1004.xlsx' #Estatico
ClaseCostos = r'KA23.xlsx' #Estatico
Budget = r'Presupuesto1004-2019Qlik.xlsx' #Estatico
ZPM028 = r'ZMM028.XLSX' #Dinamico

filename3611 = r'3611.XLSX' #Dinamico
filename2993 = r'2993.XLSX' #Dinamico

filenameGrafico = r'Grafica.xlsx'
filenameRC01 = r'RC01.xlsx'

#OT
filenameOt = r'IW39.xlsx'

In [ ]:
#Charging DataSet
fullpath_CeCo = os.path.join(filepathCostos,CeCo)
fullpath_ClCo = os.path.join(filepathCostos,ClaseCostos)
fullpath_Budget = os.path.join(filepathCostos,Budget)

fullpath_3611 = os.path.join(filepathCostos,filename3611)
fullpath_2993 = os.path.join(filepathCostos,filename2993)
fullpath_Ot = os.path.join(filepathOtAv, filenameOt)
fullpathZPM028 = os.path.join(filepathMateriales, ZPM028)
fullpathGrafico = os.path.join(filepathReporte,filenameGrafico)
fullpathRC01 = os.path.join(filepathReporte,filenameRC01)

In [ ]:
dbceco = pd.read_excel(fullpath_CeCo, converters={'CeCo':str})
dbclco = pd.read_excel(fullpath_ClCo, converters={'ClCo':str})
dbbudget = pd.read_excel(fullpath_Budget, converters={'CeCo':str, 'ClCo':str, 'MesNumero':str})
dbzpm028 = pd.read_excel(fullpathZPM028, convers ={'Material':str,
                                                   'Grupo de Artículos':str,
                                                   'Desc. Grupo de Artículos':str,
                                                   'Grupo de Compras':str})
dbzpm028 = dbzpm028[['Material', 'Desc. Grupo de Artículos',
                     'Grupo de Compras',
                     'Grupo de Artículos']].rename(columns = {'Grupo de Compras':'GrupoCompras',
                                                              'Grupo de Artículos':'GrupoArticulos', 
                                                              'Desc. Grupo de Artículos':'DescripGrupoArticulos'})

In [ ]:
def mesdenom(x):
    if x == '01':
        return 'Enero'
    elif x == '02':
        return 'Febrero'
    elif x == '03':
        return 'Marzo'
    elif x == '04':
        return 'Abril'
    elif x == '05':
        return 'Mayo'
    elif x == '06':
        return 'Junio'
    elif x == '07':
        return 'Julio'
    elif x == '08':
        return 'Agosto'
    elif x == '09':
        return 'Setiembre'
    elif x == '10':
        return 'Octubre'
    elif x == '11':
        return 'Noviembre'
    elif x == '12':
        return 'Diciembre'
    else: 
        return 'Error'

In [ ]:
#Get Budget by Days, weeks, month
budget = pd.merge(dbbudget,
                    dbceco[['CeCo','AreaResponsable']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')

# all Days between two Dates
begin = datetime.date(2019, 1, 1)
end = datetime.date(2019, 12, 31)

next_day = begin

allday = []
allweek = []
allmonth = []


while True:
    if next_day > end:
        break
    allday.append(next_day.strftime('%d'))
    allweek.append('Sem ' + next_day.strftime('%V'))
    allmonth.append(next_day.strftime('%m'))
    next_day += datetime.timedelta(days=1)

# Array of all year days
allyear = pd.DataFrame()
allyear['DiaNumero'] = allday 
allyear['SemanaNumero'] = allweek 
allyear['MesNumero'] = allmonth
allyear['DiaNumeroInt'] = [int(x) for x in allyear['DiaNumero']]
MaxTable = allyear.groupby(['MesNumero'], 
                           as_index = False).agg({'DiaNumeroInt':'max'}).rename(columns = {'DiaNumeroInt':'MesDiaMax'})
allyear = pd.merge(allyear[['DiaNumero',
                             'SemanaNumero',
                             'MesNumero',
                             'DiaNumeroInt']],
                    MaxTable[['MesNumero',
                              'MesDiaMax']],
                    on = 'MesNumero',
                    how = 'left')
allyear['MesDescrip'] = allyear['MesNumero'].apply(lambda x: mesdenom(x))

# Get Budget by NaturalezaGasto
budget = budget.groupby(['Escenario', 'CeCo', 'DenomCeCo', 'Division', 'Proceso', 'Año',
       'MesNumero', 'Mes', 'NaturalezaGasto',
       'AreaResponsable'], as_index = False).agg({'Costo':'sum'})

# Get Budget by days
budgetdays = pd.merge(allyear,
                      budget,
                      on = 'MesNumero',
                     how = 'left')

budgetdays['Costo'] = budgetdays['Costo']/budgetdays['MesDiaMax']

In [ ]:
dbot = pd.read_excel(fullpath_Ot, head = 0,converters={'Orden':str,'Aviso':str,'Centro coste':str})
dbot = dbot.rename(columns={'Texto breve':'DenomOrden',
                                'Clase de orden':'ClaseOrden',
                                'Cl.actividad PM':'ClaseActividadOrden',
                                'Status sistema':'EstatusSistema',
                                'Status usuario':'EstatusUsuario',
                                'Indicador ABC':'CriticidadEq',
                                'Fe.inic.extrema':'FechaProgramada',
                                'Hora inicio':'HoraProgramada',
                                'Fe.inicio real':'FechaInicioEjecucion',
                                'Hora in.real':'HoraInicioEjecucion',
                                'Fecha fin real':'FechaFinEjecucion',
                                'Fin real (hora)':'HoraFinEjecucion',
                                'Liberación real':'FechaDeProgramacon',
                                'Denominación':'DenomEq',
                                'Ubicac.técnica':'UbicacionTecnica',
                                'Denominación.1':'DenomUbT',
                                'Centro coste':'CeCo',
                                'Plan mant.prev.':'PlanMtto',
                                'Revisión':'Revision',
                                'Grupo planif.':'GrupoPlanif',
                                'Pto.tbjo.resp.':'PuestoTrabajoResp',
                                'Puesto trabajo':'PuestoTrabajo'})


## Data de Costos en base a OT

In [ ]:
db2993 = pd.read_excel(fullpath_2993, converters={'Orden':str,
                                                  'Clase de coste':str,
                                                  'Material':str})
db2993 = db2993.dropna(subset=['Orden'])

db2993 = db2993.rename(columns={'División':'Division',
                                'Período':'Periodo',
                                'Fe.contabilización':'FechaContable',
                                'Denominación del objeto':'DenomOrden',
                                'Valor/Moneda objeto':'Costo',
                                'Moneda del objeto':'Moneda',
                                'Texto breve de material':'DenomMaterial',
                                'Cantidad total reg.':'CantMaterial',
                                'Ud. cantidad contab.':'UnidadCantMaterial',
                                'Documento compras':'DocumentoCompras',
                                'Texto de pedido':'DenomDocCompras',
                                'Clase de coste':'ClCo',
                                'Descrip.clases coste':'DenomClCo',
                                'Nombre':'DenomProveedor',
                                'Clase de documento':'ClaseDocumento',
                                'Indic.cargo/abono':'IndCargoAbono'})
db2993['Escenario'] = 'Real'
db2993['TipoData'] = '2993'
db2993["FechaContableStr"] = db2993["FechaContable"].apply(lambda x: x.strftime('%m/%d/%Y'))
db2993["DiaNumero"] = db2993["FechaContable"].apply(lambda x: x.strftime('%d'))
db2993["SemanaNumero"] = db2993["FechaContable"].apply(lambda x: 'Sem ' + x.strftime('%V'))
db2993["MesNumero"] = db2993["FechaContable"].apply(lambda x: x.strftime('%m'))
db2993["MesDescrip"] = db2993["MesNumero"].apply(lambda x: mesdenom(x))

In [ ]:
costosot = pd.merge(db2993[['Orden', 'DenomOrden',
                            'Costo',
                            'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                            'Proveedor', 'DenomProveedor',
                            'Escenario',
                            'ClCo', 'DenomClCo',
                            'MesNumero', 'MesDescrip', 'SemanaNumero', 'DiaNumero',
                            'TipoData']],
                  dbzpm028[['Material','DescripGrupoArticulos','GrupoCompras']],
                  on='Material', 
                  how = 'left',
                  validate = 'm:1')

In [ ]:
#Reporte de Costos / 2993 <-> Clase de Coste
costosot = pd.merge(costosot, 
                    dbclco[['ClCo', 'NaturalezaGasto']], 
                    on='ClCo', 
                    how = 'left',
                    validate = 'm:1')

In [ ]:
#Reporte de Costos / 2993,clase de coste<-> Orden de Mantenimiento
costosot = pd.merge(costosot, 
                    dbot[['Orden', 'ClaseOrden', 'CriticidadEq', 'Equipo', 'DenomEq', 'UbicacionTecnica', 'DenomUbT',
                          'PlanMtto',
                          'CeCo']], 
                    on = 'Orden', 
                    how = 'left',
                    validate = 'm:1')

In [ ]:
#Reporte de Costos / 2993,clase de coste,Orden de Mantenimiento<-> Centro de Coste
costosot = pd.merge(costosot,
                    dbceco[['CeCo', 'DenomCeCo',
                            'AreaResponsable','Proceso']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')


## Data de Costos en base a CeCo

In [ ]:
db3611 = pd.read_excel(fullpath_3611, converters={'Orden partner':str,
                                                  'Centro de coste':str,
                                                  'Clase de coste':str,
                                                  'Documento compras':str,
                                                  'Proveedor':str,
                                                  'Material':str})
db3611 = db3611.dropna(subset=['Centro de coste'])
db3611 = db3611.rename(columns={'Período':'Periodo',
                                'Fe.contabilización':'FechaContable',
                                'Valor/Moneda objeto':'Costo',
                                'Moneda del informe':'Moneda',
                                'Orden partner':'Orden',
                                'Denominación del objeto del interlocutor':'DenomOrden',
                                'Documento compras':'DocumentoCompras',
                                'Texto de pedido':'DenomDocCompras',
                                'Nombre':'DenomProveedor',
                                'Centro de coste':'CeCo',
                                'Denominación del objeto':'DenomCeCo',
                                'Clase de coste':'ClCo',
                                'Descrip.clases coste':'DenomClCo',
                                'Texto breve de material':'DenomMaterial',
                                'Cantidad total reg.':'CantMaterial',
                                'Ud. cantidad contab.':'UnidadCantMaterial',
                                'Clase de documento':'ClaseDocumento',
                                'Indic.cargo/abono':'IndCargoAbono'})

db3611['Escenario'] = 'Real'
db3611['TipoData'] = '3611'
db3611["FechaContableStr"] = db3611["FechaContable"].apply(lambda x: x.strftime('%m/%d/%Y'))
db3611["DiaNumero"] = db3611["FechaContable"].apply(lambda x: x.strftime('%d'))
db3611["SemanaNumero"] = db3611["FechaContable"].apply(lambda x: 'Sem ' + x.strftime('%V'))
db3611["MesNumero"] = db3611["FechaContable"].apply(lambda x: x.strftime('%m'))
db3611["MesDescrip"] = db3611["MesNumero"].apply(lambda x: mesdenom(x))

db3611 = db3611.dropna(subset=['CeCo'])

In [ ]:
costosceco = pd.merge(db3611[['Orden', 'DenomOrden', 
                            'Costo',
                            'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                            'Proveedor', 'DenomProveedor',
                            'Escenario',
                            'CeCo' ,'DenomCeCo',
                            'ClCo', 'DenomClCo',
                            'MesNumero', 'MesDescrip', 'SemanaNumero', 'DiaNumero',
                            'TipoData']],
                  dbzpm028[['Material','DescripGrupoArticulos']],
                  on='Material', 
                  how = 'left',
                  validate = 'm:1')

In [ ]:
# Reporte de Costos / 3611 <-> Clase de Coste
costosceco = pd.merge(costosceco,
                    dbclco[['ClCo',
                            'NaturalezaGasto']],
                    on='ClCo',
                    how = 'left',
                    validate = 'm:1')

In [ ]:
#Reporte de Costos / 2993,clase de coste<-> Orden de Mantenimiento
costosceco = pd.merge(costosceco, 
                    dbot[['Orden',
                          'ClaseOrden',
                          'Equipo','DenomEq','CriticidadEq',
                          'UbicacionTecnica', 'DenomUbT', 'PlanMtto']], 
                    on='Orden', 
                    how = 'left',
                    validate = 'm:1')

In [ ]:
#Reporte de Costos / 3611,clase de coste,Orden de Mantenimiento<-> Centro de Coste
costosceco = pd.merge(costosceco, 
                    dbceco[['CeCo', 'AreaResponsable','Proceso']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')

In [ ]:
costoscecosot = costosceco[costosceco.Orden.isna()]

In [ ]:
RC01 = pd.concat([costosot[['DiaNumero', 'SemanaNumero', 'MesNumero', 'MesDescrip',
                            'Orden', 'DenomOrden','ClaseOrden', 'PlanMtto',
                            'Costo',
                            'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                            'DescripGrupoArticulos',
                            'Equipo', 'DenomEq', 'CriticidadEq',
                            'UbicacionTecnica', 'DenomUbT',
                            'Proveedor', 'DenomProveedor',
                            'Escenario', 
                            'NaturalezaGasto',
                            'CeCo', 'DenomCeCo',
                            'ClCo', 'DenomClCo',
                            'AreaResponsable',
                            'Proceso',
                            'TipoData']],
                  budgetdays[['DiaNumero',
                              'SemanaNumero',
                              'MesNumero',
                              'MesDescrip',
                              'AreaResponsable',
                              'Proceso',
                              'Escenario',
                              'NaturalezaGasto',
                              'Costo',]]], 
                 ignore_index=True, 
                 sort=True,
                 join = 'outer')

In [ ]:
RC02 = pd.concat([costosceco[['Orden', 'DenomOrden','ClaseOrden','PlanMtto',
                              'Costo',
                                'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                                'DescripGrupoArticulos',
                                'Equipo','DenomEq','CriticidadEq',
                                'UbicacionTecnica', 'DenomUbT',
                                'Proveedor', 'DenomProveedor',
                                'Escenario',
                                'AreaResponsable',
                                'NaturalezaGasto',
                                'CeCo' ,'DenomCeCo',
                                'ClCo', 'DenomClCo',
                                'MesNumero', 'MesDescrip', 'SemanaNumero', 'DiaNumero',
                                'Proceso',
                                'TipoData']], 
                  budgetdays[['DiaNumero', 'SemanaNumero', 'MesNumero', 'MesDescrip',
                              'CeCo', 'DenomCeCo',
                              'Costo',
                              'Proceso',
                              'AreaResponsable',
                              'Escenario',
                              'NaturalezaGasto']]], 
                 ignore_index=True, 
                 sort=True)

## Combinaciónd e todas las Tx de Costos

In [ ]:
#  2993 - 3611 - OT - 
RC03 = pd.concat([RC01[['Orden', 'DenomOrden', 'ClaseOrden',
                        'PlanMtto',
                        'Costo',
                        'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                        'DescripGrupoArticulos',
                        'Proveedor', 'DenomProveedor',
                        'Equipo','DenomEq','CriticidadEq',
                        'UbicacionTecnica', 'DenomUbT',
                        'Escenario',
                        'NaturalezaGasto',
                        'AreaResponsable', 'Proceso',
                        'CeCo' ,'DenomCeCo',
                        'ClCo', 'DenomClCo',
                        'MesNumero', 'MesDescrip', 'SemanaNumero', 'DiaNumero',
                        'TipoData']], 
                  costoscecosot[['Orden', 'DenomOrden',
                                'Costo',
                                'Material', 'DenomMaterial', 'CantMaterial', 'UnidadCantMaterial',
                                'DescripGrupoArticulos',
                                'Proveedor', 'DenomProveedor',
                                'Escenario',
                                'NaturalezaGasto',
                                'AreaResponsable', 'Proceso',
                                'CeCo' ,'DenomCeCo',
                                'ClCo', 'DenomClCo',
                                'MesNumero', 'MesDescrip', 'SemanaNumero', 'DiaNumero',
                                'TipoData']]], 
                 ignore_index=True, 
                 sort=True)

In [ ]:
RC03.columns

In [ ]:
# costoceco = costoceco[costoceco["Orden"].isna()]

In [ ]:
with pd.ExcelWriter(fullpathRC01, mode='w') as writer:
    #RC01.to_excel(writer, sheet_name='Sheet1',index = False)
    #RC02.to_excel(writer, sheet_name='Sheet2',index = False)
    RC03.to_excel(writer, sheet_name='Sheet3',index = False)

In [ ]:
# wb = load_workbook(fullpath_RC01)

# wsdb1 = wb['Sheet1']
# wsdb1

#RC02xl = RC02.drop(['FechaContable'], axis=1)

#for r in dataframe_to_rows(RC02, index=False, header=True):
#    wsdb1.append(r)

# wsdb2 = wb['Sheet2']

#wb.save(fullpath_RC01)

In [ ]:
os.startfile(fullpathGrafico)

In [ ]:
<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {}
}
</script>
</head>
<body>


</body>
</html>
